In [1]:
# import os
# for i in range(14):
#     os.system(f'wget https://f000.backblazeb2.com/file/malay-dataset/tatabahasa/dataset-tatabahasa-{i}.pkl')

In [2]:
import pickle

with open('dataset-tatabahasa-1.pkl', 'rb') as fopen:
    data = pickle.load(fopen)

In [3]:
data[6][0]

[['Meskipun', 2],
 ['Madonna', 2],
 ['melihat', 2],
 ['keadaan', 2],
 ['ini', 2],
 ['sebagai', 2],
 ['menggugat', 2],
 ['imejnya', 2],
 [',', 2],
 ['beliau', 2],
 ['tidak', 2],
 ['dapat', 2],
 ['mengambil', 2],
 ['tindakan', 2],
 ['undang-undang', 2],
 ['terhadap', 2],
 ['majalah-majalah', 2],
 ['tersebut', 2],
 ['kerana', 2],
 ['penerbitan', 2],
 ['dilakukan', 2],
 ['dalam', 2],
 ['cara', 2],
 ['yang', 2],
 ['dilihat', 2],
 ['sebagai', 2],
 ['di', 2],
 ['sisi', 2],
 ['undang-undang', 2],
 ['media', 2],
 ['Amerika', 2],
 ['Syarikat', 2],
 ['.', 2]]

In [4]:
y = ' '.join([w[0] for w in data[5][0]])
x = ' '.join([w[0] for w in data[5][1]])
x, y

('Tertirisnya gambar ini sangat aneh dunia memandangkan keadaannya yang terkenal ke ketika ini .',
 'Tertirisnya gambar-gambar ini sangat mengejutkan dunia memandangkan keadaannya yang terkenal pada ketika ini .')

In [5]:
from tqdm import tqdm
import re

def cleaning(string):
    string = string.replace('\n', ' ').replace('\t', ' ')
    string = re.sub(r'[ ]+', ' ', string).strip()
    return string

In [6]:
import tensorflow as tf
import tensorflow_datasets as tfds
from t5.data import preprocessors as prep
import functools
import t5
import gin
import sentencepiece as spm
from glob import glob
import os
import json

gin.parse_config_file('pretrained_models_base_operative_config.gin')
vocab = 'sp10m.cased.ms-en.model'
sp = spm.SentencePieceProcessor()
sp.Load(vocab)

True

In [7]:
pkls = glob('dataset-tatabahasa-*.pkl')

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
train_X, train_Y, test_X, test_Y = [], [], [], []
for file in pkls:
    print(file)
    with open(file, 'rb') as fopen:
        data = pickle.load(fopen)
    train, test = train_test_split(data, test_size = 500)
    for row in tqdm(train):
        y = ' '.join([w[0] for w in row[0]])
        x = ' '.join([w[0] for w in row[1]])
        train_X.append(x)
        train_Y.append(y)
        
    for row in tqdm(test):
        y = ' '.join([w[0] for w in row[0]])
        x = ' '.join([w[0] for w in row[1]])
        test_X.append(x)
        test_Y.append(y)

dataset-tatabahasa-5.pkl


100%|██████████| 500/500 [00:00<00:00, 173849.95it/s]


dataset-tatabahasa-11.pkl


100%|██████████| 500/500 [00:00<00:00, 185720.16it/s]


dataset-tatabahasa-0.pkl


...

100%|██████████| 500/500 [00:00<00:00, 186745.50it/s]


dataset-tatabahasa-13.pkl


100%|██████████| 500/500 [00:00<00:00, 191906.30it/s]


dataset-tatabahasa-3.pkl


100%|██████████| 500/500 [00:00<00:00, 188457.23it/s]


In [11]:
len(train_X), len(test_X)

(1391940, 7000)

In [12]:
with tf.io.gfile.GFile('kesalahan-tatabahasa.tsv', "w") as outfile:
    for i in tqdm(range(len(train_X))):
        if len(train_X) and len(train_Y):
            l = cleaning(train_X[i])
            r = cleaning(train_Y[i])
            outfile.write("%s\t%s\n" % (l, r))

100%|██████████| 1391940/1391940 [00:31<00:00, 44248.76it/s]


In [13]:
def kesalahan_dataset(split, shuffle_files = False):
    del shuffle_files
    ds = tf.data.TextLineDataset(
        [
            'kesalahan-tatabahasa.tsv'
        ]
    )

    ds = ds.map(
        functools.partial(
            tf.io.decode_csv,
            record_defaults = ['', ''],
            field_delim = '\t',
            use_quote_delim = False,
        ),
        num_parallel_calls = tf.data.experimental.AUTOTUNE,
    )
    ds = ds.map(lambda *ex: dict(zip(['question', 'answer'], ex)))
    return ds

def kesalahan_preprocessor(ds):
    def to_inputs_and_targets(ex):
        return {
            'inputs': tf.strings.join(['kesalahan tatabahasa: ', ex['question']]),
            'targets': ex['answer'],
        }

    return ds.map(
        to_inputs_and_targets,
        num_parallel_calls = tf.data.experimental.AUTOTUNE,
    )

In [14]:
t5.data.TaskRegistry.remove('kesalahan_dataset')
t5.data.TaskRegistry.add(
    'kesalahan_dataset',
    dataset_fn = kesalahan_dataset,
    splits = ['train'],
    text_preprocessor = [kesalahan_preprocessor],
    sentencepiece_model_path = vocab,
    metric_fns = [t5.evaluation.metrics.accuracy],
)


In [15]:
nq_task = t5.data.TaskRegistry.get("kesalahan_dataset")
ds = nq_task.get_dataset(split='paraphrase.tsv', sequence_length={"inputs": 1024, "targets": 1024})
r = tfds.as_numpy(ds)

In [16]:
next(r)

{'inputs_plaintext': b'kesalahan tatabahasa: Editornya ialah Onn Bin Jaafar ( 1930 - 33 ) , Sayyid Alwi bin Sayyid Shaykh al-Hadi ( 1933 - 34 ) , Sayyid Hussein bin Ali Alsagoff ( 1934 - 1941 ) , dan Cikgu Ibrahim Yaakob ( 1941 ) .',
 'inputs': array([ 1946, 31301,    31,  6456,    38,   358,   544,   152,  3514,
        17579,    13,     4,  8847,    13,     7,  3403,    13,     5,
           13,    14, 10871,   128,  1400,   418,  1922,  1890, 10871,
          128,  1400, 26379,  3484,  1781,     7,  3884,   387,    13,
            4, 20254,    13,     7,  3472,    13,     5,    13,    14,
        10871,   128,  1400,  8060,  1890,  1612,   418,    16,  5524,
         1696,    13,     4, 20911,    13,     7, 14390,    13,     5,
           13,    14,    22,  6904,  1956,  2884, 26976,    13,     4,
        14390,    13,     5,    13,     3,     1]),
 'targets_plaintext': b'Editornya ialah Onn Bin Jaafar ( 1930 - 33 ) , Sayyid Alwi bin Sayyid Shaykh al-Hadi ( 1933 - 34 ) , Sayyid Huss

In [17]:
with open('test-set-kesalahan-tatabahasa.json', 'w') as fopen:
    json.dump({'X': test_X, 'Y': test_Y}, fopen)